In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import subprocess
import os
import sys
sys.path.append('/usr/workspace/vanover1/approx-llvm/approx')
from approx_modules import approx

from sklearn.ensemble import ExtraTreesRegressor

/usr/tce/packages/python/python-3.7.2/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/usr/tce/packages/python/python-3.7.2/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecatio

In [2]:
N_KERNELS = 3
KERNEL_NAMES = ["result", "s1", "x"]

In [3]:
def run_simpsons(endpoint):
    
    # compile and run simpsons instrumented with HPAC directives
    # need to use /usr/tce/bin/git binary because the default git version grabbed by this subprocess is too early for the --show-superproject-working-tree flag
    subprocess.check_call("rm -f test.h5 && source $(/usr/tce/bin/git rev-parse --show-superproject-working-tree)/scripts/activate_hpac_env.sh && make simpsons-hpac && ./simpsons-hpac {}".format(endpoint), shell=True)

    # open database
    approxDataProfile = approx.approxApplication("./test.h5")
    
    # get output
    Y = approxDataProfile.getApplicationOutput()['area'][0]

    # get aggregated kernel outputs
    kernel_outputs = []
    for kernel_name in KERNEL_NAMES:
        kernel_outputs.append(np.mean([approxDataProfile[region_name].Y().mean() for region_name in approxDataProfile.getRegionNames() if region_name.startswith(kernel_name)]))
    
    return kernel_outputs, Y

In [4]:
# define neighborhood for simpsons analysis
center = 1.0
eps = 0.05
n_sample = 2**6

# uniform sample of n_sample endpoints from the neighborhood
XX = np.linspace(center-eps, center+eps, n_sample)

In [5]:
# run simpsons
kernel_outs = []
YY = []
for n, X in enumerate(XX):
    temp = run_simpsons(X)
    kernel_outs.append(temp[0])
    YY.append(temp[1])
kernel_outs = np.array(kernel_outs)
YY = np.array(YY)

In [6]:
# analyze kernel output sensitivity
et = ExtraTreesRegressor(n_estimators=100,
                                    criterion="mse",
                                    # max_features=int(round(XX.shape[1] / 3)),
                                    max_depth=8,
                                    min_samples_split=2,
                                    min_samples_leaf=max(1, int(round(np.sqrt(kernel_outs.shape[0]) / np.sqrt(1000)))),
                                    min_weight_fraction_leaf=0,
                                    max_leaf_nodes=None,
                                    #bootstrap=True,
                                    #oob_score=True,
                                    random_state=1)
et.fit(kernel_outs, YY)
for k_no in range(N_KERNELS):
    print("{} : {}".format(KERNEL_NAMES[k_no], et.feature_importances_[k_no]))

result : 0.04140328995909101
s1 : 0.9221339425397423
x : 0.036462767501166955


## Compare to:
```
=== BEGIN ADAPT REPORT ===
6000009 total independent/intermediate variables
1 dependent variables
Mixed-precision recommendation:
  Replace variable b           max error introduced: 0.000000e+00  count: 1           totalerr: 0.000000e+00
  Replace variable a           max error introduced: 0.000000e+00  count: 1           totalerr: 0.000000e+00
  Replace variable h           max error introduced: 4.152677e-15  count: 1           totalerr: 4.152677e-15
  Replace variable pi          max error introduced: 9.154282e-14  count: 1           totalerr: 9.569550e-14
  Replace variable result      max error introduced: 2.967209e-11  count: 2000002     totalerr: 2.976779e-11
  DO NOT replace   x           max error introduced: 2.397519e-07  count: 2000001     totalerr: 2.397817e-07
  DO NOT replace   s1          max error introduced: 8.160601e-05  count: 2000002     totalerr: 8.184579e-05
=== END ADAPT REPORT ===
```